In [1]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
from bs4 import BeautifulSoup

In [2]:
dataset = pd.read_csv("/home/sietsmarj/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/test_results_genes_auc_exonic.csv", header=0, index_col=0)
dataset

,#Chrom,Pos,Ref,Alt,max_AF,Consequence,label,revel,clinpred,sift,provean,PHRED,fathmm_score,capice,ponp2,gene,auc,exonic
0,21,33974174,C,G,0.000058,STOP_LOST,LB/B,NaN,NaN,NaN,NaN,15.050,NaN,0.003275,NaN,C21orf59,NaN,1
1,21,33974278,C,CA,0.000045,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.304741,NaN,C21orf59,NaN,1
2,X,99661625,G,C,0.000037,SYNONYMOUS,LB/B,NaN,NaN,0.00,-0.00,0.806,NaN,0.000075,NaN,PCDH19,0.906593,1
3,X,99663098,G,GT,0.000013,STOP_GAINED,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.998694,NaN,PCDH19,0.906593,1
4,X,99662275,C,G,0.000212,NON_SYNONYMOUS,LB/B,0.527,NaN,0.95,2.51,27.000,NaN,0.863940,0.436,PCDH19,0.906593,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10540,22,36662041,AATAATT,A,0.141400,INFRAME,LP/P,NaN,NaN,NaN,8.98,7.508,NaN,0.004829,NaN,APOL1,NaN,1
10541,2,31754497,T,C,0.000033,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,19.790,0.951270,0.918898,NaN,SRD5A2,0.921053,0
10542,2,31754393,C,T,0.000000,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,21.700,NaN,0.985064,NaN,SRD5A2,0.921053,0
10543,2,31758741,T,C,0.000200,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,20.800,0.912937,0.474717,NaN,SRD5A2,0.921053,0


In [3]:
# Loading in the original test_results to see if SNV's are missing

original_dataset = pd.read_csv("/home/sietsmarj/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/test_results.txt", sep='\t', header=0)
original_dataset

,#Chrom,Pos,Ref,Alt,max_AF,Consequence,label,revel,clinpred,sift,provean,PHRED,fathmm_score,capice,ponp2
0,21,33974174,C,G,0.000058,STOP_LOST,LB/B,NaN,NaN,NaN,NaN,15.050,NaN,0.003275,NaN
1,X,99661625,G,C,0.000037,SYNONYMOUS,LB/B,NaN,NaN,0.000,-0.0,0.806,NaN,0.000075,NaN
2,17,29509638,C,T,0.000000,SYNONYMOUS,LB/B,NaN,NaN,0.647,-0.0,10.760,NaN,0.001511,NaN
3,21,35742999,C,T,0.000133,SYNONYMOUS,LB/B,NaN,NaN,0.000,-0.0,18.640,NaN,0.000621,NaN
4,1,2160973,G,A,0.000000,SYNONYMOUS,LB/B,NaN,NaN,0.000,-0.0,16.340,0.008252,0.000012,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10837,17,29556342,G,A,0.000000,SYNONYMOUS,LP/P,NaN,NaN,0.000,-0.0,11.450,NaN,0.000139,NaN
10838,11,5248177,A,T,0.000344,SYNONYMOUS,LP/P,NaN,NaN,0.000,-0.0,17.970,NaN,0.069934,NaN
10839,15,48787324,T,C,0.000000,SYNONYMOUS,LP/P,NaN,NaN,0.664,-0.0,12.170,0.019530,0.909190,NaN
10840,19,17947957,G,A,0.000015,SYNONYMOUS,LP/P,NaN,NaN,0.000,-0.0,10.990,NaN,0.001238,NaN


In [4]:
# Merging 
missing_o_c = pd.merge(dataset, original_dataset, how='outer', indicator=True)
print(f"Missing number of samples: {missing_o_c[missing_o_c['_merge'] == 'right_only'].shape[0]}")
print(f"Genes that are present in original, but not in processed: {missing_o_c[missing_o_c['_merge'] == 'right_only']['gene'].unique()}")

Missing number of samples: 1935
Genes that are present in original, but not in processed: [nan]


In [5]:
# Deleting the variables
del(missing_o_c, original_dataset)

In [6]:
list_of_hereditary_auto_inflammatory_disorder_mutations = []

In [7]:
page_request = requests.get("https://infevers.umai-montpellier.fr/web/index.php")
page_request.close()

In [8]:
soup = BeautifulSoup(page_request.content, 'html.parser')

In [9]:
navbar = soup.find(id='myNavbar')

In [10]:
# Getting the genes.

for link in navbar.find_all('a'):
    get_text = link.get_text().strip()
    if len(get_text) > 0:
        list_of_hereditary_auto_inflammatory_disorder_mutations.append(get_text)

# This is just personal, ignore please

In [52]:
url = 'https://azerty.nl/product/amd/3940002/ryzen-9-3900x-processor-3-8-ghz-4-6-ghz-'

In [53]:
page_request = requests.get(url)
page_request.close()

In [67]:
soup = BeautifulSoup(page_request.content, 'html.parser')
# page_request.cookies['__Secure-TnetID']
detail = soup.find(id='detail')
price = 0
for item in detail.find_all('meta'):
    get_text = item.get_text().strip()
    if get_text.startswith("€ "):
        price = float(get_text.split('€ ')[1].split('\n')[0].replace(",", "."))
        break


# Continue

In [68]:
# Loading in the omim gene list and cleaning it.

test_omim = pd.read_csv("/home/sietsmarj/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/genemap2.txt", header=0, sep='\t', skiprows=3)
test_omim = test_omim[test_omim['Genomic Position Start'].notnull()]
test_omim['# Chromosome'] = test_omim['# Chromosome'].apply(lambda x: x.split("chr")[1])
test_omim = test_omim[['# Chromosome', 'Genomic Position Start', 'Genomic Position End', 'Gene Symbols']]
test_omim.columns = ['chr','start','end','gene']
initial_omim_list = list(test_omim['gene'].unique())
actual_omim_list = []
reset_timer = time.time()
done = 0
total = len(initial_omim_list)
for gene in initial_omim_list:
    time_fls = time.time()
    if time_fls - reset_timer > 10:
        print(f"Still processing, done: {round(done/total*100, ndigits=2)}%")
        reset_timer = time.time()
    gene_split = gene.split(", ")
    if len(gene_split) > 1:
        for g in gene_split:
            if g not in actual_omim_list:
                actual_omim_list.append(g)
    else:
        actual_omim_list += gene_split
    done += 1


In [71]:
# Comparing it to the gene list that I currently have.

dataset['in_omim'] = 0

current_gene_list = list(dataset['gene'].unique())
reset_timer = time.time()
done = 0
total = len(actual_omim_list)
in_list = 0
actually_in_dataset = []
for gene in actual_omim_list:
    time_fls = time.time()
    if time_fls - reset_timer > 10:
        print(f"Still processing, done: {round(done/total*100, ndigits=2)}%")
        reset_timer = time.time()
    if gene in current_gene_list:
        actually_in_dataset.append(gene)
        dataset.loc[dataset[dataset['gene'] == gene].index, 'in_omim'] = 1
        in_list += 1
    done += 1
print(f"Total percentage of OMIM loci in dataset: {round(in_list/total*100, ndigits=2)}%")
dataset

Total percentage of OMIM loci in dataset: 4.67%


,#Chrom,Pos,Ref,Alt,max_AF,Consequence,label,revel,clinpred,sift,provean,PHRED,fathmm_score,capice,ponp2,gene,auc,exonic,in_omim
0,21,33974174,C,G,0.000058,STOP_LOST,LB/B,NaN,NaN,NaN,NaN,15.050,NaN,0.003275,NaN,C21orf59,NaN,1,1
1,21,33974278,C,CA,0.000045,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.304741,NaN,C21orf59,NaN,1,1
2,X,99661625,G,C,0.000037,SYNONYMOUS,LB/B,NaN,NaN,0.00,-0.00,0.806,NaN,0.000075,NaN,PCDH19,0.906593,1,1
3,X,99663098,G,GT,0.000013,STOP_GAINED,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.998694,NaN,PCDH19,0.906593,1,1
4,X,99662275,C,G,0.000212,NON_SYNONYMOUS,LB/B,0.527,NaN,0.95,2.51,27.000,NaN,0.863940,0.436,PCDH19,0.906593,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10540,22,36662041,AATAATT,A,0.141400,INFRAME,LP/P,NaN,NaN,NaN,8.98,7.508,NaN,0.004829,NaN,APOL1,NaN,1,1
10541,2,31754497,T,C,0.000033,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,19.790,0.951270,0.918898,NaN,SRD5A2,0.921053,0,1
10542,2,31754393,C,T,0.000000,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,21.700,NaN,0.985064,NaN,SRD5A2,0.921053,0,1
10543,2,31758741,T,C,0.000200,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,20.800,0.912937,0.474717,NaN,SRD5A2,0.921053,0,1


In [73]:
# A lot of entries are 1 for in_omim, just check the ones that aren't.

dataset[dataset['in_omim'] == 0]

,#Chrom,Pos,Ref,Alt,max_AF,Consequence,label,revel,clinpred,sift,provean,PHRED,fathmm_score,capice,ponp2,gene,auc,exonic,in_omim
177,X,24380945,C,G,0.001621,NONCODING_CHANGE,LB/B,NaN,NaN,NaN,NaN,9.197,NaN,0.000502,NaN,SUPT20HL1,NaN,1,0
959,X,105280707,GATGCTGGAAGCC,G,0.000233,INFRAME,LB/B,NaN,NaN,NaN,25.11,11.910,NaN,0.395947,NaN,SERPINA7,NaN,1,0
960,X,105280503,C,T,0.000052,NON_SYNONYMOUS,LB/B,0.363,NaN,0.998,7.85,22.800,NaN,0.004191,0.402,SERPINA7,NaN,1,0
961,X,105280627,T,A,0.000082,NON_SYNONYMOUS,LB/B,0.350,NaN,0.986,3.38,21.800,NaN,0.001321,0.356,SERPINA7,NaN,1,0
962,X,105280568,G,A,0.000157,NON_SYNONYMOUS,LB/B,0.310,NaN,0.933,3.74,17.280,NaN,0.000191,0.455,SERPINA7,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10149,17,33434045,G,A,0.000102,STOP_GAINED,LP/P,NaN,NaN,NaN,NaN,37.000,0.101396,0.109137,NaN,RAD51L3-RFFL,NaN,1,0
10150,17,33433425,G,A,0.000411,STOP_GAINED,LP/P,NaN,NaN,NaN,NaN,39.000,NaN,0.906808,NaN,RAD51L3-RFFL,NaN,1,0
10151,17,33430565,T,C,0.000000,CANONICAL_SPLICE,LP/P,NaN,NaN,NaN,NaN,23.600,0.991828,0.781858,NaN,RAD51L3-RFFL,NaN,1,0
10438,2,86071665,G,A,0.000067,STOP_GAINED,LP/P,NaN,NaN,NaN,NaN,36.000,NaN,0.777764,NaN,ST3GAL5,0.954545,0,0


In [75]:
# Lots of entries with a 1 in "in_omim", making sure that this is true.

done = 0
total = len(current_gene_list)
for gene in current_gene_list:
    if gene in actual_omim_list:
        done+=1
print(f"Percentage of current dataset are actually omim genes: {round(done/total*100, ndigits=2)}%")

Percentage of current dataset are actually omim genes: 96.73%


In [77]:
# Let's see where the dataset is not in omim and we have an AUC

dataset[(dataset['in_omim'] == 0) & (dataset['auc'].notnull())]

# Not a lot.

,#Chrom,Pos,Ref,Alt,max_AF,Consequence,label,revel,clinpred,sift,provean,PHRED,fathmm_score,capice,ponp2,gene,auc,exonic,in_omim
3292,2,71186142,C,T,0.000401,UPSTREAM,LB/B,NaN,NaN,NaN,NaN,5.635,0.284608,0.000090,NaN,ATP6V1B1,0.976190,0,0
3293,2,71190419,C,G,0.000065,NON_SYNONYMOUS,LP/P,0.962,NaN,1.0,6.67,27.800,NaN,0.728767,0.913,ATP6V1B1,0.976190,0,0
4610,7,70254732,A,G,0.000009,CANONICAL_SPLICE,LB/B,NaN,NaN,NaN,NaN,33.000,NaN,0.490748,NaN,AUTS2,0.937500,0,0
5853,1,92729270,C,T,0.000009,STOP_GAINED,LB/B,NaN,NaN,NaN,NaN,45.000,NaN,0.220031,NaN,GLMN,0.796875,0,0
5854,1,92732038,ACT,A,0.000030,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,34.000,NaN,0.753025,NaN,GLMN,0.796875,0,0
5855,1,92730115,AATGAC,A,0.000004,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.761056,NaN,GLMN,0.796875,0,0
5856,1,92763550,C,CA,0.000058,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,26.100,NaN,0.586644,NaN,GLMN,0.796875,0,0
5857,1,92729242,A,AG,0.000009,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.207089,NaN,GLMN,0.796875,0,0
5858,1,92752046,CT,C,0.000030,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,29.000,NaN,0.825738,NaN,GLMN,0.796875,0,0
5859,1,92733508,GGTAAA,G,0.000032,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,34.000,NaN,0.885047,NaN,GLMN,0.796875,0,0


In [82]:
dataset.drop('in_omim', axis=1, inplace=True)

In [80]:
dataset['in_inflammatory'] = 0

In [83]:
# Comparing the inflammatory gnes to the gene list that I currently have.

current_gene_list = list(dataset['gene'].unique())
reset_timer = time.time()
done = 0
total = len(list_of_hereditary_auto_inflammatory_disorder_mutations)
in_list = 0
actually_in_dataset = []
for gene in list_of_hereditary_auto_inflammatory_disorder_mutations:
    time_fls = time.time()
    if time_fls - reset_timer > 10:
        print(f"Still processing, done: {round(done/total*100, ndigits=2)}%")
        reset_timer = time.time()
    if gene in current_gene_list:
        actually_in_dataset.append(gene)
        dataset.loc[dataset[dataset['gene'] == gene].index, 'in_inflammatory'] = 1
        in_list += 1
    done += 1
print(f"Total percentage of Inflammatory genes in dataset: {round(in_list/total*100, ndigits=2)}%")
dataset

Total percentage of Inflammatory genes in dataset: 23.26%


,#Chrom,Pos,Ref,Alt,max_AF,Consequence,label,revel,clinpred,sift,provean,PHRED,fathmm_score,capice,ponp2,gene,auc,exonic,in_inflammatory
0,21,33974174,C,G,0.000058,STOP_LOST,LB/B,NaN,NaN,NaN,NaN,15.050,NaN,0.003275,NaN,C21orf59,NaN,1,0
1,21,33974278,C,CA,0.000045,FRAME_SHIFT,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.304741,NaN,C21orf59,NaN,1,0
2,X,99661625,G,C,0.000037,SYNONYMOUS,LB/B,NaN,NaN,0.00,-0.00,0.806,NaN,0.000075,NaN,PCDH19,0.906593,1,0
3,X,99663098,G,GT,0.000013,STOP_GAINED,LB/B,NaN,NaN,NaN,NaN,35.000,NaN,0.998694,NaN,PCDH19,0.906593,1,0
4,X,99662275,C,G,0.000212,NON_SYNONYMOUS,LB/B,0.527,NaN,0.95,2.51,27.000,NaN,0.863940,0.436,PCDH19,0.906593,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10540,22,36662041,AATAATT,A,0.141400,INFRAME,LP/P,NaN,NaN,NaN,8.98,7.508,NaN,0.004829,NaN,APOL1,NaN,1,0
10541,2,31754497,T,C,0.000033,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,19.790,0.951270,0.918898,NaN,SRD5A2,0.921053,0,0
10542,2,31754393,C,T,0.000000,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,21.700,NaN,0.985064,NaN,SRD5A2,0.921053,0,0
10543,2,31758741,T,C,0.000200,NONCODING_CHANGE,LP/P,NaN,NaN,NaN,NaN,20.800,0.912937,0.474717,NaN,SRD5A2,0.921053,0,0


In [84]:
# Just checking how much of my dataset is from the inflammatory genes.

done = 0
total = len(current_gene_list)
for gene in current_gene_list:
    if gene in list_of_hereditary_auto_inflammatory_disorder_mutations:
        done+=1
print(f"Percentage of current dataset are actually inflammatory genes: {round(done/total*100, ndigits=2)}%")

Percentage of current dataset are actually omim genes: 0.58%
